In [ ]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime

warnings.filterwarnings("ignore")

hoy = datetime.now()
año_actual = hoy.year
mes_actual = hoy.month

if mes_actual == 1:
    mes_anterior = 12
    año_anterior = año_actual - 1
else:
    mes_anterior = mes_actual - 1
    año_anterior = año_actual

mes_anterior_str = (hoy - pd.DateOffset(months=1)).strftime('%b').upper()[:3] # DIC
mes_actual_str = hoy.strftime('%b').upper()[:3] # ENE

mes_año_anterior = f"{mes_anterior_str}{str(año_anterior)[2:]}" # DIC23
mes_año_actual = f"{mes_actual_str}{str(año_actual)[2:]}" # ENE24

fecha_anterior = f"{año_anterior}{str(mes_anterior).zfill(2)}" # 202312
fecha_actual = f"{año_actual}{str(mes_actual).zfill(2)}" # 202401

asignacion_1 = f"bases/asignacion/{fecha_anterior}/base_asignacion_{mes_año_anterior}.xlsx"
asignacion_2 = f"bases/asignacion/{fecha_actual}/base_asignacion_{mes_año_actual}.xlsx"

cruce_asignacion_path = f"bases/asignacion/cruce_asignacion_{mes_año_anterior}-{mes_año_actual}.xlsx"

print(asignacion_1)
print(asignacion_2)

In [11]:
df_asig_1 = pd.read_excel(asignacion_1, usecols=['CODIGO', 'ID_FLAG', 'AGENCIA_CORRECTA'])
df_asig_1['CODIGO'] = df_asig_1['CODIGO'].astype('Int64').astype(str).str.zfill(8)
df_asig_1.rename(columns={'ID_FLAG': 'FLAG', 'AGENCIA_CORRECTA': 'AGENCIA'}, inplace=True)

df_asig_2 = pd.read_excel(asignacion_2, usecols=['CODIGO', 'FLAG', 'AGENCIA'])
df_asig_2['CODIGO'] = df_asig_2['CODIGO'].astype('Int64').astype(str).str.zfill(8)

df_asig_1.groupby('CODIGO').agg({'FLAG': 'first'}).reset_index().sort_values('CODIGO', ascending=False, inplace=True)
df_asig_2.groupby('CODIGO').agg({'FLAG': 'first'}).reset_index().sort_values('CODIGO', ascending=False, inplace=True)

In [15]:
df_asig = pd.merge(df_asig_1, df_asig_2, on='CODIGO', how='outer', suffixes=('_1', '_2'))

df_asig[['FLAG_1', 'FLAG_2']] = df_asig[['FLAG_1', 'FLAG_2']].fillna(0).astype(int)

conditions = [
    (df_asig['FLAG_1'] == df_asig['FLAG_2']) | ((df_asig['FLAG_1'] > 1) & (df_asig['FLAG_2'] > 1)),
    (df_asig['FLAG_1'] == 1) & (df_asig['FLAG_2'] > 1),
    (df_asig['FLAG_1'] > 1) & (df_asig['FLAG_2'] == 1),
    (df_asig['FLAG_1'] == 0) & (df_asig['FLAG_2'] == 1),
    (df_asig['FLAG_1'] == 0) & (df_asig['FLAG_2'] > 1),
    (df_asig['FLAG_1'] == 1) & (df_asig['FLAG_2'] == 0),
    (df_asig['FLAG_1'] > 1) & (df_asig['FLAG_2'] == 0)
]
choices = [
    'IGUAL', 'MONO -> MULTI', 'MULTI -> MONO', 'NULL -> MONO', 'NULL -> MULTI', 'MONO -> NULL', 'MULTI -> NULL'
]
df_asig['FLAG'] = np.select(conditions, choices, default='NULL')

df_asig['AGENCIA_1'] = df_asig['AGENCIA_1'].fillna('SIN AGENCIA').astype(str).str.upper()
df_asig['AGENCIA_2'] = df_asig['AGENCIA_2'].fillna('SIN AGENCIA').astype(str).str.upper()
df_asig['AGENCIA'] = np.where(df_asig['AGENCIA_1'] == df_asig['AGENCIA_2'], df_asig['AGENCIA_1'], df_asig['AGENCIA_1'] + ' -> ' + df_asig['AGENCIA_2'])

df_asig = df_asig[['CODIGO', 'FLAG_1', 'FLAG_2', 'FLAG', 'AGENCIA_1', 'AGENCIA_2', 'AGENCIA']]

In [16]:
df_asig.to_excel(cruce_asignacion_path, index=False)